In [1]:
from __future__ import division
from os import set_inheritable
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import numpy as np
import random
from gsheetcoms import GSH
import pandas as pd
import time

In [2]:
##########################################
# Default Values
#########################################
_WORKBOOK = 'RoutingModel'
_TEMP_SKU_DICT = {}

##########################################
# Googel Sheet Communication / Fetch Tables as Dataframes
#########################################
_GSH = GSH()
print('Fetching data...')
_ASSUMPTIONS = _GSH.generateDataframe(_WORKBOOK, 'Assumptions')
_ORDERS = _GSH.generateDataframe(_WORKBOOK, 'Orders')
_SKUS = _GSH.generateDataframe(_WORKBOOK, 'SKUs')
_ORDERS_SKUS = _GSH.generateDataframe(_WORKBOOK, 'Orders_SKUs')
_LAYOUT = _GSH.generateDataframe(_WORKBOOK, 'Layout')

##########################################
# General Assumptions
#########################################
print('Setting Assumptions...')
_STABILIZE_TIME = int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Stabilize Time'].iloc[0]['Value'])
_PICKING_TIME = int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picking Time'].iloc[0]['Value'])
_PICKER_SPEED = int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picker Speed'].iloc[0]['Value'])
_PICKER_COST = int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picker Cost'].iloc[0]['Value'])
_STABILIZE_TIME = int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Stabilize Time'].iloc[0]['Value'])

##########################################
# Layout Specific Assumptions
#########################################
_X_LIMIT= _ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'X Limit'].iloc[0]['Value']
_X_MIDDLE= _ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'X Middle'].iloc[0]['Value']

print('Data Fetched Successfully')

Initializing Google Spreadheet Connector: gsheet.json
Fetching data...
Setting Assumptions...
Data Fetched Successfully


In [3]:
def cellInfo(cell_id):
    cell_id = str(cell_id)
    info = {
        'Cell_ID' : cell_id,
        'SKU_ID' : _LAYOUT.loc[_LAYOUT['Cell_ID'] == cell_id].iloc[0]['SKU_ID'],
        'x'  : _LAYOUT.loc[_LAYOUT['Cell_ID'] == cell_id].iloc[0]['X'],
        'y'  : _LAYOUT.loc[_LAYOUT['Cell_ID'] == cell_id].iloc[0]['Y'],
        'direction'  : _LAYOUT.loc[_LAYOUT['Cell_ID'] == cell_id].iloc[0]['Direction']
    }
    return info

In [4]:
def get_weight(sku):
    sku = str(sku)
    resp = _SKUS.loc[_SKUS['SKU_ID'] == sku].iloc[0]['Weight']
    return resp

In [5]:
def distance_calculator(cell1, cell2):
    distance = 0
    try:
        pos1 = cellInfo(cell1)
        pos2 = cellInfo(cell2)
        # Start Point
        x1 = int(pos1['x'])
        y1 = int(pos1['y'])
        direction1 = int(pos1['direction'])

        # End Point
        x2 = int(pos2['x'])
        y2 = int(pos2['y'])
        direction2 = int(pos2['direction'])

        # Distance y-axis
        distance_y = abs(y2 - y1)

        # Distance x-axis
        distance_x = 0
        x_low = 1
        x_passage = int(_X_MIDDLE)
        x_limit = int(_X_LIMIT)

        # If locations are in same corridor:
        if y1 == y2:
            distance_x = abs(x2 - x1)
        else:
            if direction1 == 1:
                if x1 <= x_passage:
                    if direction1 == direction2:
                        if x2 <= x_passage:
                            distance_x = x_passage - x1 + x_passage + x2
                        else:
                            distance_x = x2 - x1
                    else:
                        if x2 <= x_passage:
                            distance_x = x_passage - x1 + x_passage - x2
                        else:
                            distance_x = x_limit - x1 + x_limit - x2		
                else:
                    if direction1 == direction2:
                        if x2 <= x_passage:
                            distance_x = x_limit - x1 + x_limit + x2
                        else:
                            distance_x = x_limit - x1 + x2
                    else:
                        distance_x = x_limit - x1 + x_limit - x2
            else:
                if x1 <= x_passage:
                    if direction1 == direction2:
                        if x2 <= x_passage:
                            distance_x = x1 + x_passage + x_passage - x2

                        else:
                            distance_x = x1 + x_limit + x_limit - x2
                    else:
                            distance_x = x1 + x2
                else:
                    if direction1 == direction2:
                        if x2 <= x_passage:
                            distance_x = x1 - x2
                        else:
                            distance_x = x1 + x_limit - x2
                    else:
                        if x2 <= x_passage:
                            distance_x = x1 + x2
                        else:
                            distance_x = x1 - x_passage + x2 - x_passage
            ############

        # Total distance
        distance = distance_x + distance_y

    except:
        distance = 0

    finally:
        return distance

In [6]:

def total_cost():

    total_cost = 0

    # Repleneshing Cost
    repleneshing_cost = 0
    for index, sku in _SKUS.iterrows():
        time_spent = 0
        if int(sku['N_Pallets_Needed']) > 1:
            # Froklift up, down and repleneshing
            time_spent += int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Forklift Time'].iloc[0]['Value']) * 2 # Forklift Up and Down
            time_spent += int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Pallet Replenish Time'].iloc[0]['Value'])
            time_spent = time_spent / 60 # Mins to hours

            # Forklift Operator moving to postion and back 
            cell1 = '0' #initial position
            cell2 =  str(_LAYOUT.loc[_LAYOUT['SKU_ID'] == sku['SKU_ID']].iloc[0]['Cell_ID']) #sku position
            distance = distance_calculator(cell1, cell2) * 2 / 1000 #back and forth and in km
            time_spent = distance * int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picker Speed'].iloc[0]['Value'])
            repleneshing_cost += time_spent * int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Forklift Operator Cost'].iloc[0]['Value'])
    print(f'Repleneshing Cost: {repleneshing_cost}')

    # Picking Pallet Preparation
    _ORDERS['Pallet_Qty'] = _ORDERS['Pallet_Qty'].apply(pd.to_numeric)
    picking_setup_cost = int(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picking Setup Time'].iloc[0]['Value']) * (_ORDERS['Pallet_Qty'].sum() / 60)
    print(f'Picking Pallet Setup Cost: {picking_setup_cost}')


    total_cost += repleneshing_cost + picking_setup_cost
    return total_cost

print(f'Total Cost: {total_cost()}')


Repleneshing Cost: 6273.899999999998
Picking Pallet Setup Cost: 56.0
Total Cost: 6329.899999999998


In [7]:
# Define alternative N-Routes fitness function for minimization problem (fitness = count)
def route_min_cost(state_input):
    
    cost = 0
    try:
        # Prevent Duplicate states
        unique = np.unique(state_input)
        if len(unique) < len(state_input):
            cost = 10^12

        state = state_input
        #insert initial operator position in state, each state value is the sku ID, and ID = 0 is the inital position
        state = np.insert(state, 0, 0, axis=0)

        #insert final operator position in state, each state value is the sku ID, and ID = 0 is the final position
        state = np.append(state, 0)

        # Initialize time tracker (h)
        time = 0

        # Initialize BU counter
        bu_counter = 0
        BU = []

        for i in range(1, len(state) - 1):
            # Calculate travel disctance time
            if i == 1:
                sku1 = 0
                sku2 = int(_TEMP_SKU_DICT[state[i]])
            elif i == len(state):
                sku2 = 0
                sku1 = int(_TEMP_SKU_DICT[state[i-1]])
            else:
                sku1 = int(_TEMP_SKU_DICT[state[i-1]] )
                sku2 = int(_TEMP_SKU_DICT[state[i]] )

            time += (float(distance_calculator(sku1,sku2)) / 1000) / _PICKER_SPEED

            # Add picking time
            time += _PICKING_TIME / 3600

            # Check for weight adjustment
            if i < len(state) and i >  1:
                if (get_weight(sku2) > get_weight(sku1)):
                    time += _STABILIZE_TIME / 60

            # How many BUs?
            bu = _SKUS.loc[_SKUS['SKU_ID'] == str(sku2)].iloc[0]['BU']
            if bu not in BU:
                bu_counter += 1
                BU.append(bu)            

        # Operator Cost
        cost += float(time * _PICKER_COST)

        # Despicking Costs
        despicking_time = 0
        if bu_counter == 1:
            despicking_time += 0
        elif bu_counter == 2:
            despicking_time += 4.6
        elif bu_counter == 3:
            despicking_time += 6.7
        elif bu_counter >= 4:
            despicking_time += 10.2

        dispicker_cost = float(_ASSUMPTIONS.loc[_ASSUMPTIONS['Assumption'] == 'Picker Cost'].iloc[0]['Value'])
        cost += (despicking_time / 3600) * dispicker_cost
    
    except:
        pass
    
    return cost



In [8]:
def runOptimization(order_id):
    start = time.time()

    print("Defining Optimization Problem... ")
    n_states = int(_ORDERS.loc[_ORDERS['Order_ID'] == order_id].iloc[0]['NDif_Pos'])
    
    # Turn positions into integers and Define initial state
    skus_in_order_df = _ORDERS_SKUS.loc[_ORDERS_SKUS['Order_ID'] == order_id]
    init_state = []
    #_TEMP_SKU_DICT = {}
    for i in range(0, n_states):
        init_state.append(i)
        _TEMP_SKU_DICT[i] = skus_in_order_df.iloc[i]['SKU_ID']
    init_state = np.array(init_state)
    print(f'Initial State: {init_state}')
    
    # Initialize custom fitness function object
    print("Creating Minimization Function... ")
    custom_fitness = mlrose.CustomFitness(route_min_cost)

    # Initialize optimization problem
    problem = mlrose.DiscreteOpt(length = n_states, fitness_fn = custom_fitness, maximize = False, max_val = n_states)
    print(f'Route Optimization for Order ID: {order_id}')


    # Define decay schedule
    schedule = mlrose.ExpDecay()
    print("Solving Optimization Problem...")
    # Solve problem using simulated annealing
    best_state, best_fitness = mlrose.simulated_annealing(problem, schedule = schedule,  max_attempts = 100, max_iters = 10000, init_state = init_state, random_state = 1)
    #best_state, best_fitness = mlrose.genetic_alg(problem, mutation_prob = 0.2,  max_attempts = 1, max_iters = 1, random_state = 1)

    # Get SKU order from State array
    best_state_skus = []
    for item in best_state:
        best_state_skus.append(_TEMP_SKU_DICT[item])
    
    end = time.time()

    print('Problem Solved Successfully!')
    print(f'Best State: {best_state_skus}')
    print(f'Min Cost (€): {best_fitness}')
    print(f'Elapsed Time: {(end - start)}')
    return best_state_skus, best_fitness


In [272]:
best_state_skus, best_fitness = runOptimization('1056')

Defining Optimization Problem... 
Initial State: [0 1 2 3 4 5 6 7 8 9]
Creating Minimization Function... 
Route Optimization for Order ID: 1056
Solving Optimization Problem...
Problem Solved Successfully!
Best State: ['6021568', '7388408', '6937571', '5895717', '7414199', '6021577', '6968014', '6393009', '5238849', '6968112']
Min Cost (€): 1.7605555555555559
Elapsed Time: 51.921380043029785


In [ ]:
# Run all orders
for index, row in _ORDERS.iterrows():
    print('####################################')
    print(f"Order ID: {row['Order_ID']}")
    print('####################################')
    if row['NDif_SKUs'] == '1':
        sku_id = _ORDERS_SKUS.loc[_ORDERS_SKUS['Order_ID'] == row['Order_ID']].iloc[0]['SKU_ID']
        # result = _GSH.updateCell(_WORKBOOK, 'Orders', index + 2, 8, sku_id)
        row['Route_Order'] = f'[{sku_id}]'
        _TEMP_SKU_DICT[0] = sku_id
        cost = route_min_cost([0])
        # result = _GSH.updateCell(_WORKBOOK, 'Orders', index + 2, 9, str(cost))
        row['Cost'] = cost
        print('Problem Solved Successfully!')
        print(f'Best State: {sku_id}')
        print(f'Min Cost (€): {cost}')
    else:
        best_state, best_fitness = runOptimization(row['Order_ID'])
        #_GSH.updateCell(_WORKBOOK, 'Orders', index + 2, 8, str(best_state))
        row['Route_Order'] = str(best_state)
        #_GSH.updateCell(_WORKBOOK, 'Orders', index + 2, 9, str(best_fitness))
        row['Cost'] = str(best_fitness)


####################################
Order ID: 1067
####################################
Problem Solved Successfully!
Best State: 6393009
Min Cost (€): 0.19777777777777777
####################################
Order ID: 744
####################################
Defining Optimization Problem... 
Initial State: [0 1]
Creating Minimization Function... 
Route Optimization for Order ID: 744
Solving Optimization Problem...
Problem Solved Successfully!
Best State: ['2211915', '7095790']
Min Cost (€): 0.0
Elapsed Time: 0.6618220806121826
####################################
Order ID: 750
####################################
Defining Optimization Problem... 
Initial State: [0 1]
Creating Minimization Function... 
Route Optimization for Order ID: 750
Solving Optimization Problem...
Problem Solved Successfully!
Best State: ['2211982', '5691213']
Min Cost (€): 0.3355555555555556
Elapsed Time: 0.9309558868408203
####################################
Order ID: 1061
####################################


Problem Solved Successfully!
Best State: ['5945085', '7105463', '6968112', '6393009', '6464430']
Min Cost (€): 0.0
Elapsed Time: 0.4568450450897217
####################################
Order ID: 1068
####################################
Defining Optimization Problem... 
Initial State: [0 1 2 3 4]
Creating Minimization Function... 
Route Optimization for Order ID: 1068
Solving Optimization Problem...
Problem Solved Successfully!
Best State: ['6968113', '6814052', '6814053', '5962602', '7167225']
Min Cost (€): 0.0
Elapsed Time: 0.42750072479248047
####################################
Order ID: 1069
####################################
Defining Optimization Problem... 
Initial State: [0 1 2 3 4]
Creating Minimization Function... 
Route Optimization for Order ID: 1069
Solving Optimization Problem...
Problem Solved Successfully!
Best State: ['5895632', '7017345', '5934282', '7184412', '6814057']
Min Cost (€): 0.0
Elapsed Time: 0.47090792655944824
####################################
Order I

Problem Solved Successfully!
Best State: ['6851067', '6581275', '6649458', '6582990', '6968113', '6968014', '7287176', '7071705']
Min Cost (€): 0.0
Elapsed Time: 0.410916805267334
####################################
Order ID: 1063
####################################
Defining Optimization Problem... 
Initial State: [0 1 2 3 4 5 6 7]
Creating Minimization Function... 
Route Optimization for Order ID: 1063
Solving Optimization Problem...
Problem Solved Successfully!
Best State: ['7388417', '7184413', '5958776', '6581275', '5005144', '5962602', '6968113', '6153824']
Min Cost (€): 0.0
Elapsed Time: 0.4274160861968994
####################################
Order ID: 1118
####################################
Defining Optimization Problem... 
Initial State: [0 1 2 3 4 5 6 7]
Creating Minimization Function... 
Route Optimization for Order ID: 1118
Solving Optimization Problem...
Problem Solved Successfully!
Best State: ['6581275', '5467663', '6851067', '5734372', '5945085', '5934280', '7071705'

/usr/local/lib/python3.9/site-packages/mlrose/algorithms.py:311: RuntimeWarning: overflow encountered in exp
  prob = np.exp(delta_e/temp)


Problem Solved Successfully!
Best State: ['5895717', '6928446', '6851067', '6581275', '7388417', '6924270', '5947303', '6153824', '6464430', '6021577']
Min Cost (€): 0.0
Elapsed Time: 29.679270029067993
####################################
Order ID: 1117
####################################
Defining Optimization Problem... 
Initial State: [0 1 2 3 4 5 6 7 8 9]
Creating Minimization Function... 
Route Optimization for Order ID: 1117
Solving Optimization Problem...
Problem Solved Successfully!
Best State: ['7388408', '6929000', '5947126', '7388405', '6968014', '7105624', '7388410', '6153824', '6464430', '6581275']
Min Cost (€): 0.0
Elapsed Time: 30.35762310028076
####################################
Order ID: 867
####################################
Defining Optimization Problem... 
Initial State: [ 0  1  2  3  4  5  6  7  8  9 10]
Creating Minimization Function... 
Route Optimization for Order ID: 867
Solving Optimization Problem...
Problem Solved Successfully!
Best State: ['7439782', '

...

Problem Solved Successfully!
Best State: ['6182354', '5718357', '6577347', '5718357', '7257426', '4704539', '6398241', '7328025', '7363461', '7404118', '6781967', '4886486', '6536213', '5553145', '5679398', '7261923', '7327758', '6700258', '4490846', '4913140', '4704539', '5776241', '2507420', '6252532', '5630043', '7408372', '4925364', '4716888', '5453638', '6787649', '7201331', '5562119', '7094127', '2607166', '6850010', '5468982', '4383272', '5553145', '4034323', '7147180', '7109616', '5964945', '5468982', '7201331', '4716888', '6458581', '2182823', '7333825', '5494869', '6837867', '6457772', '2507420', '2214009', '4913140', '7291007', '7371929', '7411090', '5148306', '7291114', '7435451', '6744532', '6536213', '6917631', '4913140', '7411090', '5718357', '4373632', '5553145', '6182354', '4704539', '5564190', '7327758', '5564190', '7297581', '7257426', '7195209', '5373132', '2211915', '6182354', '7400532', '7297581', '5594637', '7302324', '5467123', '7095790', '6182354', '4373632', '

Problem Solved Successfully!
Best State: ['7156595', '7404072', '5070019', '6102132', '2582339', '7418020', '7375890', '7335489', '7408326', '6917655', '2907937', '7447351', '6182354', '7324619', '6909440', '5782115', '6523058', '5054401', '6878571', '7375890', '2224292', '2844079', '6306840', '6219158', '5005065', '6089206', '4734165', '6898290', '7028696', '6923901', '6881807', '7095059', '5249162', '5538920', '5791199', '5241123', '7408322', '6772136', '6881807', '7313627', '2798180', '6649053', '5241123', '2907539', '7105624', '5373132', '5054401', '6837867', '4424318', '2582339', '4734165', '5538920', '7164297', '7375890', '7408322', '6703121', '7156595', '5241123', '5844336', '7252642', '4734157', '2907539', '5054401', '7375890', '7156595', '5782115', '7302335', '7324619', '7068840', '7418020', '6628813', '6523058', '5562119', '6136781', '2798180', '6900269', '7390518', '6219158', '7363517', '6898285', '6136781', '6089206', '6763228', '5445818', '6329011', '6628813', '7302335', '

Problem Solved Successfully!
Best State: ['6494092', '7297556', '7356352', '7095890', '4034323', '3002456', '6787662', '6986608', '7303602', '3052233', '5273119', '5696985', '5373132', '7297556', '3658082', '7356352', '7328025', '5712058', '6028285', '6787662', '7207957', '4864373', '7327758', '3002456', '7334024', '7120788', '7334029', '7421347', '7404118', '7421347', '4872518', '5606345', '6252532', '7165900', '7302327', '5005065', '4750959', '7293008', '4872518', '7334029', '6290826', '6923800', '7030440', '7335491', '5360608', '7391561', '7327758', '4916018', '5328357', '4034323', '7334029', '7334029', '2607166', '3002456', '4020304', '7163255', '6494092', '5005065', '5593796', '7328025', '5373133', '7335491', '7430640', '5117294', '6494092', '6457772', '4373632', '7297556', '4882816', '5373133', '5730946', '5562119', '5005065', '7072880', '6290826', '5593788', '3002456', '4882816', '6536213', '5595421', '7297556', '7363331', '3002456', '7072880', '3052221', '3690249', '6846661', '

Problem Solved Successfully!
Best State: ['7443608', '6028285', '7070555', '7201073', '6949002', '5712058', '5249122', '6592516', '5746161', '2798180', '6949004', '4490751', '4738009', '6028285', '5322836', '7070555', '6868811', '5608351', '2632478', '5249122', '7363123', '4383272', '4916018', '5712058', '6344858', '2879958', '7408497', '6435492', '6948998', '6435492', '5005065', '6089518', '6986608', '7375889', '7164607', '5328357', '7071705', '7291114', '5005065', '7408497', '5768468', '2540497', '7164399', '7327751', '7124252', '7308156', '4916018', '4734159', '6452022', '6949002', '7408497', '7408497', '2689516', '5712058', '7105624', '5724071', '7443608', '5328357', '6759678', '6868811', '2474413', '7327751', '7291007', '5872559', '7443608', '6457772', '3002456', '6028285', '5595752', '2474413', '2430285', '5562119', '5328357', '5135637', '5768468', '7316047', '5712058', '5595752', '7068840', '7333826', '6028285', '5201596', '5712058', '5135637', '7364619', '7156595', '2907953', '

Problem Solved Successfully!
Best State: ['6986608', '7334029', '5872559', '3002456', '6453385', '6881807', '7201073', '6539956', '5872560', '6881819', '7371936', '7141309', '7327751', '7334029', '7269036', '7201331', '3002456', '4916018', '6986608', '5712058', '6787662', '5273133', '5779701', '6881807', '3881459', '6923901', '5791700', '6881819', '7164399', '6881819', '6634435', '6416545', '6047457', '2211953', '2879958', '4750959', '7070555', '5791700', '6634435', '2798180', '5793825', '5090397', '5562119', '7418218', '7291007', '3690249', '5779701', '7439647', '6925840', '7439647', '2798180', '7095205', '4915932', '6881807', '7404118', '5751607', '6772827', '6416545', '7408497', '5272801', '7163251', '7418218', '5305520', '7395734', '6772827', '7141309', '7327751', '7334029', '7430640', '7163251', '2582339', '7291007', '4750959', '6536213', '5793825', '4750959', '6881807', '7430640', '6182354', '6582982', '7334029', '7356352', '6881807', '7411092', '5779701', '6649053', '6329011', '